# ASL Action Recognition

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/03-emotion-analysis.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/03-emotion-analysis.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/03-emotion-analysis.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Start EVA server
We are reusing the start server notebook for launching the EVA server

In [1]:
!wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

File ‘00-start-eva-server.ipynb’ already there; not retrieving.



AccessDenied: (pid=53095, name='eva_server')

AccessDenied: (pid=53095, name='eva_server')

In [36]:
# An ASL video of the action computer
!wget -nc https://gatech.box.com/shared/static/io36vhiznhbls1ogi4jw994phv72e7mt.mp4 -O computer_asl.mp4

#ASL UDF
!wget -nc https://raw.githubusercontent.com/georgia-tech-db/eva/master/eva/udfs/asl_action_recognition.py -O asl_action_recognition.py 

#Pickle file containing the mapping of the ASL action to index
!wget -nc https://raw.githubusercontent.com/georgia-tech-db/eva/master/eva/udfs/asl_20_actions_map.pkl -O asl_20_actions_map.pkl

### Adding the video file to EVADB for analysis

In [37]:
cursor.execute('DROP TABLE ASL_ACTIONS')
response = cursor.fetch_all()
print(response)
cursor.execute('LOAD VIDEO "computer_asl.mp4" INTO ASL_ACTIONS;')
response = cursor.fetch_all()
print(response)

### Visualize the video

In [39]:
from IPython.display import Video
Video("computer_asl.mp4", height=450, width=800, embed=True)

### Create an user-defined function(UDF) for analyzing the frames

In [44]:
cursor.execute("DROP UDF ASLActionRecognition")
response = cursor.fetch_all()
print(response)
cursor.execute("""CREATE UDF IF NOT EXISTS ASLActionRecognition
        INPUT  (Frame_Array NDARRAY UINT8(3, 16, 224, 224))
        OUTPUT (labels NDARRAY STR(ANYDIM))
        TYPE  Classification
        IMPL  'asl_action_recognition.py'
        """)
response = cursor.fetch_all()
print(response)

### Run the UDF on the Video to find the ASL action

In [45]:
cursor.execute("""SELECT FIRST(id), ASLActionRecognition(SEGMENT(data)) 
                  FROM ASL_ACTIONS 
                  SAMPLE 5
                  GROUP BY '16f';""")
response = cursor.fetch_all()
print(response)

In [46]:
import cv2
from pprint import pprint
from matplotlib import pyplot as plt

def annotate_video(detections, input_video_path, output_video_path):
    color1=(207, 248, 64)
    color2=(255, 49, 49)
    thickness=4

    vcap = cv2.VideoCapture(input_video_path)
    width = int(vcap.get(3))
    height = int(vcap.get(4))
    fps = vcap.get(5)
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v') #codec
    video=cv2.VideoWriter(output_video_path, fourcc, fps, (width,height))

    frame_id = 0
    label = detections['aslactionrecognition.labels'][0]
    # Capture frame-by-frame
    # ret = 1 if the video is captured; frame is the image
    ret, frame = vcap.read() 

    while ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        x1, y1, x2, y2 = 350, 100, 900, 500
        # object bbox
        frame=cv2.rectangle(frame, (x1, y1), (x2, y2), color1, thickness) 
        # object label
        cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color1, thickness)
        
        # frame label
        cv2.putText(frame, 'Frame ID: ' + str(frame_id), (700, 500), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color2, thickness) 
    
        video.write(frame)
        # Show every twentieth frame
        if frame_id % 20 == 0:
            plt.imshow(frame)
            plt.show()
    
        frame_id+=1
        ret, frame = vcap.read()

    video.release()
    vcap.release()

In [47]:
input_path = 'computer_asl.mp4'
output_path = 'computer_annotated.mp4'

dataframe = response.batch.frames
annotate_video(dataframe, input_path, output_path)